In [1]:
import os

%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow/research'

In [2]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/mlflow'

## 1. Update `config.yaml`

```yaml
data_validation:
  root_dir: artifacts/data_validation
  unzip_data_path: artifacts/data_ingestion/winequality-red.csv
  status_file_path: artifacts/data_validation/status.txt
```

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


## 2. Update `schema.yaml`

```yaml
columns:
  fixed acidity: float64
  volatile acidity: float64
  citric acid: float64
  residual sugar: float64
  chlorides: float64
  free sulfur dioxide: float64
  total sulfur dioxide: float64
  density: float64
  pH: float64
  sulphates: float64
  alcohol: float64
  quality: int64

target:
  name: quality
```

## 4. Update the Entity

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    unzip_data_path: Path
    status_file_path: Path
    data_schema: dict

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [8]:
from mlflow_project.constants import *
from mlflow_project.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
        schema_file_path: Path = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        """
        Creates the root directory and returns
        the configuration for data validation.

        Returns:
            DataValidationConfig: Configuration for data validation.
        """
        data_validation = self.config.data_validation
        schema = self.schema.columns

        create_directories([data_validation.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(data_validation.root_dir),
            unzip_data_path=Path(data_validation.unzip_data_path),
            status_file_path=Path(data_validation.status_file_path),
            data_schema=dict(schema),
        )

        return data_validation_config

## 6. Update the Components

In [9]:
from mlflow_project import logger


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validation_columns(self) -> bool:
        """Validates the data schema and data types.

        Raises:
            e: Exception.

        Returns:
            bool: False if validation failed else True.
        """
        try:
            validation_status = None

            df = pd.read_csv(self.config.unzip_data_path)
            columns_datatypes = df.dtypes.to_dict()
            data_schema = self.config.data_schema

            for col, dtype in columns_datatypes.items():
                # If column not in schema, validation failed
                if col not in data_schema:
                    validation_status = False
                    with open(self.config.status_file_path, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                    return validation_status
                # If column in schema, then proceed to check data type and if not the same, validation failed
                elif dtype != data_schema[col]:
                    validation_status = False
                    with open(self.config.status_file_path, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                    return validation_status

            # After everything, validation success
            validation_status = True
            with open(self.config.status_file_path, "w") as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            logger.exception(e)
            raise e

## 7. Update the Pipeline

In [10]:
try:
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validation_columns()
except Exception as e:
    raise e

[ 2023-10-24 21:05:55,159 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-10-24 21:05:55,162 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-10-24 21:05:55,164 ] 33 common mlflow_project -  INFO - Loaded YAML file successfully from: schema.yaml
[ 2023-10-24 21:05:55,165 ] 53 common mlflow_project -  INFO - Created directory at: artifacts
[ 2023-10-24 21:05:55,165 ] 53 common mlflow_project -  INFO - Created directory at: artifacts/data_validation
